# Однослойный перцептрон на TensorFlow
### (База изображений MNIST)
База MNIST (Mixed National Institue of Standards and Technology) - база изображений рукописных цифр, часто используется в исследовательских целях в задачах обработки изображений и машинного обучения (еще несколько лет назад).

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.metrics import confusion_matrix
import pandas as pd
import sys

# Скачиваеем данные, сохраняем на диск. 
DATA_DIR = '/tmp/data' if not 'win32' in sys.platform else "c:\\tmp\\data"

# Загружаем данные 
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

print("Число изобажений в обучающем множестве: {}".format(len(data.train.images)))
print("Число меток {}.".format(len(data.train.labels)))

Отрисуем несколько примеров

Начнем с одного изображения, затем отрисуем сразу целый пакет.

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline  

IMAGE_IX_IN_DATASET = 0

first_img = data.train.images[IMAGE_IX_IN_DATASET].reshape(28, 28)
first_lbl = data.train.labels[IMAGE_IX_IN_DATASET].argmax()

plt.figure()
plt.imshow(first_img, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Вообще-то это должно быть {}".format(first_lbl))

Отобразим сразу 100 цифр на одном графике (10x10).

Для лучшего восприятия мы отобразим *инвертируем* изображения. 

In [ ]:
N_IMAGES = 10
img = np.vstack(
    [
        np.hstack(
            [img.reshape(28, 28) for img in data.train.images[np.random.choice(1000, N_IMAGES)]]
        )  for i in range(N_IMAGES)
    ]
)
img = np.logical_not(img)

plt.figure()
plt.imshow(img, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("100 случайных цифр")

# Перцептрон с Softmax функцией

Построим однослойный перцептрон, распознающий цифры. На вход он будет принимать яркости пикселей наших изображений, и выдавать наружу номер класса (цифру).

(Каждый пиксель анализируется отдельно, никакой пространственной информации перцептрон учитывать не будет)

$ u_j = \sum_i x_{ji} w_{ji} +b_j$ 

Ответ будем формировать по схеме "победитель забирает всё": будем считать ответом сети (номером класса) номер нейрона, из которого вышел максимальный отклик.

$$
y_j = softmax(u_j) = \frac{e^{u_j}}{\sum_{i=0}^9 e^{u_i}}
$$

In [ ]:
# Посмотрим на "сырые" данные
img = data.train.images[:1000]

# Plot 
plt.figure()
plt.imshow(np.logical_not(img).T, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Каждая колонка - это одно развернутое в вектор изображение...")

Не слишком заметны закономерности - проделаем еще две операции:
    1. выберем только центры изображений;
    2. отсортируем по ответам.
    
В итоге получается более понятная структура, возможно, что человек после некоторой тренировки мог бы научиться распознавать цифры в таком виде. 

In [ ]:

N_IMAGES = 1000

# Вырежем центры
center_img = [img.reshape(28, 28)[8:22, 8:22].ravel() 
              for img in  data.train.images[:N_IMAGES]]

# сортируем изображения
sorted_lbls = np.argsort(data.train.labels.argmax(axis=1)[:N_IMAGES])
center_img = np.array(center_img)[sorted_lbls]

# Plot 
plt.figure()
plt.imshow(np.logical_not(center_img).T, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Каждая колонка - центр изображения...")

Построим нашу модель.

Потребуется определить:
    1. placehodler
    2. Variable
    3. Размерости [None, 784];
    4. matmul 

In [ ]:
# Начинаем строить модель
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W) + b

Требуется функция потерь.

In [ ]:
cross_entropy = \
    tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

Воспользуемся градиентным спуском

In [ ]:
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Нам нужно оценить точность модели:
The final elements we need are measures of how well our model is preforming.
    1. correct_mask: маркер, правильно сеть ответила или нет;
    2. accuracy: доля правильных ответов.

In [ ]:
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

На этом шаге определен вычислительный граф. Создаем сессию и запускаем модель.

In [ ]:
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

with tf.Session() as sess:

    # Обучение
    sess.run(tf.global_variables_initializer())
    for _ in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})

    # Тестирование
    is_correct, acc, y_true_vec, y_pred_vec = sess.run([correct_mask, accuracy, y_true, y_pred], 
                               feed_dict={x: data.test.images, y_true: data.test.labels})

print("Accuracy: {:.4}%".format(acc*100))


# Какие изображения не распознаны?

Посмотрим, какие цифры распознаны верно, а какие нет.

In [ ]:
is_correct

In [ ]:
correct_ix = np.where(is_correct)[0]
correct_img = data.train.images[correct_ix]

N_IMAGES = 10
img = np.vstack([np.hstack([img.reshape(28, 28) 
                            for img in correct_img[np.random.choice(len(correct_ix), N_IMAGES)]])
                 for i in range(N_IMAGES)])
img = np.logical_not(img)

plt.figure()
plt.imshow(img, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Правильно разпознано!")

In [ ]:
incorrect_ix = np.where(np.logical_not(is_correct))[0]
incorrect_img = data.train.images[incorrect_ix]

N_IMAGES = 10
img = np.vstack([np.hstack([img.reshape(28, 28) 
                            for img in incorrect_img[np.random.choice(len(incorrect_ix), N_IMAGES)]])
                 for i in range(N_IMAGES)])
img = np.logical_not(img)

plt.figure()
plt.imshow(img, cmap='gray')
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title("Ошибки!")

Построим матрицу ошибок:

In [ ]:
conf_mat = confusion_matrix(y_true_vec.argmax(axis=1), y_pred_vec.argmax(axis=1))

print(pd.DataFrame(conf_mat))